In [50]:
from IPython.display import display
from IPython.display import clear_output
from IPython.display import HTML
from IPython.display import Javascript
from ipywidgets import widgets
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import feedparser
import pymongo
from pymongo import MongoClient


cliente = MongoClient()
db = cliente.test_database
noticias = db.noticias

dropDownSites = widgets.Dropdown(options=[u'El País',u'ElDiario',u'Público', u'El Mundo'], value='ElDiario',
                                 description='Elige un medio: ', disabled=False, button_style='info')
textBox = widgets.Text()
label = widgets.Label(value='Introduzca la palabra clave:',)
button = widgets.Button(description="Buscar")
containerRequest = widgets.VBox(children=[dropDownSites,label, textBox,button])
container = None
textRSS = None
fuente = None

pd.options.display.max_colwidth = 200

def apply_Style():
    Javascript("""tabla = document.getElementsByTagName(\"td\");
                for(var i=0; i < tabla.length; i++){
                tabla[i].style.background= '#D3F1F1'; tabla[i].style.textAlign = 'center'; tabla[i].style.height = '100px !important';}
                var encabezados = document.getElementsByTagName(\"th\");
                for(var i=0; i < encabezados.length; i++){ encabezados[i].style.textAlign = 'center';encabezados[i].style.background = '#58AFFB';} 
                var tablaEntera = document.getElementsByTagName(\"table\");
                for(var i=0; i < tablaEntera.length; i++){ tablaEntera[i].style.offsetWidth = '800px !important';}""")

def handle_submit(sender):
    global textRSS
    etiqueta = None
    if(dropDownSites.value == u'ElDiario'):
        textRSS = feedparser.parse('http://www.eldiario.es/rss/')
        etiqueta = 'summary'
        fuente = 'ElDiario'
    elif (dropDownSites.value == u'El País'):
        textRSS = feedparser.parse('http://ep00.epimg.net/rss/sociedad/portada.xml')
        etiqueta = 'summary'
        fuente = 'El Pais'
    elif (dropDownSites.value == u'Público'):
        textRSS = feedparser.parse('http://www.publico.es/rss/')
        #print textRSS['entries'][1]['summary_detail']['value']
        etiqueta = 'description'
        fuente = 'Publico'
    elif(dropDownSites.value == u'El Mundo'):
        textRSS = feedparser.parse('http://www.elmundo.es/rss/portada.xml')
        etiqueta = 'summary'
        fuente ='ElMundo'
    titulares = []
    textos = []
    fechas = []
    autores = []
    for entry in textRSS['entries']:
        try:
            soup = BeautifulSoup(entry[etiqueta], "lxml")
            if(textBox.value != ''):
                for palabra in soup.text.split(' '):
                    if(textBox.value in palabra):            
                        titulares.append(entry['title'])
                        fechas.append(entry['published'])
                        autores.append(entry['author'])
                        if(fuente != 'ElMundo'):
                            textos.append(soup.text)
                        else:
                            noticiaSoup = feedparser.parse(entry['links'][0]['href'])
                            if(BeautifulSoup(noticiaSoup['feed']['summary'])):
                                noticiaSoup = BeautifulSoup(noticiaSoup['feed']['summary'])
                            if(noticiaSoup.find('article')):
                                textos.append(noticiaSoup.find('article').text)
                                

                            
            else:
                titulares.append(entry['title'])
                fechas.append(entry['published'])
                autores.append(entry['author'])
                if(fuente != 'ElMundo'):
                    textos.append(soup.text)
                else:
                    noticiaSoup = feedparser.parse(entry['links'][0]['href'])
                    if(BeautifulSoup(noticiaSoup['feed']['summary'])):
                        noticiaSoup = BeautifulSoup(noticiaSoup['feed']['summary'])
                    if(noticiaSoup.find('article')):
                        textos.append(noticiaSoup.find('article').text)
                        
                    
        except:
            pass
    idInsertadas = [None]*len(titulares)
    for titular in titulares:
        indice = titulares.index(titular)
        autor = autores[indice]
        fecha = fechas[indice]
        texto = textos[indice].encode('utf-8').decode('utf-8')
        noticia = { 'author': autor, 'title':titular,
                    'publishDate':fecha, 'text':texto, 'source':'RSS'}
        idInsertadas[titulares.index(titular)]= noticias.update_one( {'title':titular},{ '$set': {'author': autor, 'title':titular,
                    'publishDate':fecha, 'text':texto, 'source':fuente}}, upsert=True)
    df = pd.DataFrame({'titular':titulares, 'texto':textos, 'fecha':fechas, 'autor':autores})
    df = df.reset_index(drop=True)
    htmlDF = df.to_html(escape=False)
    htmlTabla = widgets.HTML(value= htmlDF)
    global container 
    if container is not None:
        container.close()
    container = widgets.VBox(children=[htmlTabla])
    display(container)
    display(Javascript("""                var tabla = document.getElementsByTagName(\"td\");
                for(var i=0; i < tabla.length; i++){
                tabla[i].style.background= '#D3F1F1'; tabla[i].style.textAlign = 'center'; tabla[i].style.height = '100px !important';}
                var encabezados = document.getElementsByTagName(\"th\");
                for(var i=0; i < encabezados.length; i++){ encabezados[i].style.textAlign = 'center';encabezados[i].style.background = '#58AFFB';} 
                var tablaEntera = document.getElementsByTagName(\"table\");
                for(var i=0; i < tablaEntera.length; i++){ tablaEntera[i].style.offsetWidth = '800px !important';}"""))
    
button.on_click(handle_submit)
display(containerRequest)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>